<a href="https://colab.research.google.com/github/sujatabhandari/Practice/blob/main/Bhandaris_Final_Project_CMPS325.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sujata Bhandari**

**CMPS 325**

**Final Project**



# Project One

This project involves observing the progress of several different network architectures learning an image classification task.

Note that each network does require the cells before Part One to have been run, but they do not require the networks before them to have run. So you can break up your work and not have to run all the networks just to re-run the last one.

You should set the runtime type to GPU under the Runtime menu to speed the computations. Note that this may restrict you to having one notebook open at a time.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


See a List of TensorFlow datasets at: https://knowyourdata-tfds.withgoogle.com/

See how to load them:
https://www.tensorflow.org/datasets/overview

See data augmentation discussion: https://www.tensorflow.org/tutorials/images/data_augmentation

In [ ]:
# TensorFlow
import tensorflow as tf

# Datasets
import tensorflow_datasets as tfds

In [ ]:
# Load in the tf_flowers dataset
# Separate into training, testing and validation sets

train_split = 'train[:80%]'
validation_split = 'train[80%:90%]'
test_split = 'train[90%:]'
(ds_train, ds_validation, ds_test), ds_info = tfds.load(
    'tf_flowers',
    split=[train_split, validation_split, test_split],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print(ds_info)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]


Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='tf_flowers',
    version=3.0.1,
    description='A large set of images of flowers',
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=5),
    }),
    total_num_examples=3670,
    splits={
        'train': 3670,
    },
    supervised_keys=('image', 'label'),
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
    redistribution_info=,
)



In [ ]:
# Set some network parameters

# Number of classes in the dataset
num_classes = ds_info.features['label'].num_classes
# Size of each image dimension (width and height)
img_size = 80
# Size of a batch of images
batch_size = 128

# Function to apply to each image to prepare it for training and inference
def process_img(image, label):
  # Resize the images, pad if needed to preserve aspect ratio
  #image = tf.image.resize_with_pad(image, img_size, img_size)
  image = tf.image.resize(image, (img_size, img_size))
  # Change from ints in range 0-255 to floats in range 0-1
  image = tf.cast(image, tf.float32) / 255.
  # Change label from integer to one-hot vector
  label = tf.one_hot(label, num_classes)
  return image, label

# Setup data pipeline
# Training images should be shuffled
train = ds_train.map(
    process_img, 
    num_parallel_calls=tf.data.AUTOTUNE
    ).cache().shuffle(ds_info.splits[train_split].num_examples).batch(128).prefetch(tf.data.AUTOTUNE)

# Validation data for tuning hyperparameters
validation = ds_validation.map(
    process_img, 
    num_parallel_calls=tf.data.AUTOTUNE
    ).cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Test data for final evaluation
test = ds_test.map(
    process_img, 
    num_parallel_calls=tf.data.AUTOTUNE
    ).cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)

## Part One - Basic Two-Layer Network

Build the network and the train the model using the cells below.

In [ ]:
# Basic two-layer network
two_layer_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(192, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax'),
])
two_layer_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
# Train the model
# Watch the training and validation accuracy over time
# A notably higher training accuracy means overfitting
epochs=10
history = two_layer_model.fit(
  train,
  validation_data=validation,
  epochs=epochs
)

Epoch 1/10
23/23 [==============================] - 0s 17ms/step - loss: 1.4154 - categorical_accuracy: 0.4278 - val_loss: 1.3027 - val_categorical_accuracy: 0.4387
Epoch 2/10
23/23 [==============================] - 0s 16ms/step - loss: 1.5841 - categorical_accuracy: 0.4016 - val_loss: 1.4479 - val_categorical_accuracy: 0.3787
Epoch 3/10
23/23 [==============================] - 0s 16ms/step - loss: 1.2639 - categorical_accuracy: 0.4728 - val_loss: 1.3360 - val_categorical_accuracy: 0.4332
Epoch 4/10
23/23 [==============================] - 0s 16ms/step - loss: 1.3226 - categorical_accuracy: 0.4472 - val_loss: 1.3221 - val_categorical_accuracy: 0.4441
Epoch 5/10
23/23 [==============================] - 0s 16ms/step - loss: 1.2451 - categorical_accuracy: 0.4663 - val_loss: 1.3076 - val_categorical_accuracy: 0.4496
Epoch 6/10
23/23 [==============================] - 0s 17ms/step - loss: 1.2218 - categorical_accuracy: 0.4843 - val_loss: 1.3104 - val_categorical_accuracy: 0.4687
Epoch 7/10

In [ ]:
# How big is this model?
two_layer_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 19200)             0         
                                                                 
 dense_9 (Dense)             (None, 192)               3686592   
                                                                 
 dense_10 (Dense)            (None, 64)                12352     
                                                                 
 dense_11 (Dense)            (None, 5)                 325       
                                                                 
Total params: 3,699,269
Trainable params: 3,699,269
Non-trainable params: 0
_________________________________________________________________


### Questions
1. How did the network do? What were the final training and validation accuracies?
**The netwrok did not do very well as the training and validation accuracies were not that high. The accuracy for final training was about 52% whereas the accuracy for validation was about 42%.**
2. Is there evidence of overfitting?
**Yes, there is a clear evidence of overfitting as my training accuracy was higher than my validation accuracy from Epoch2 through Epoch6. The training accuracies from Epoch 2 to Epoch 6 were higher(40%, 47%,44.7%,46%,and 48%)respectively compared to their validation accuracies of(37%, 43%,44.4%,45%,and 47%)respectively.**
3. Why do you think the network had trouble? What are the limitations of this architecture?
**I think that the network had trouble because the data was non-parametric or non-linear. Similarly, I think that the basic two-layer network did not work well for this training model with the data. Some of the limitations of this architecture are:
1) Slower to train the model
2) Despite of it running slow, there can be seen overfitting in the results that are not helpful at all.**

## Part Two - Basic Convolutional Network

Build the network and the train the model using the cells below.

In [ ]:
# A basic convolutional neural network with two sets of convolution/pooling layers
# Notice that typically only one hidden dense layer is used
basic_conv_model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(24, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(192, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax'),
])
basic_conv_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
# Train the model
# Watch the training and validation accuracy over time
# A notably higher training accuracy means overfitting
epochs=10
history = basic_conv_model.fit(
  train,
  validation_data=validation,
  epochs=epochs
)

Epoch 1/10
23/23 [==============================] - 2s 42ms/step - loss: 1.7366 - categorical_accuracy: 0.3505 - val_loss: 1.2650 - val_categorical_accuracy: 0.4768
Epoch 2/10
23/23 [==============================] - 1s 31ms/step - loss: 1.2572 - categorical_accuracy: 0.4731 - val_loss: 1.1282 - val_categorical_accuracy: 0.5777
Epoch 3/10
23/23 [==============================] - 1s 32ms/step - loss: 1.1104 - categorical_accuracy: 0.5644 - val_loss: 1.0433 - val_categorical_accuracy: 0.6049
Epoch 4/10
23/23 [==============================] - 1s 32ms/step - loss: 1.0179 - categorical_accuracy: 0.6063 - val_loss: 1.0797 - val_categorical_accuracy: 0.5559
Epoch 5/10
23/23 [==============================] - 1s 31ms/step - loss: 0.9375 - categorical_accuracy: 0.6379 - val_loss: 1.1958 - val_categorical_accuracy: 0.5014
Epoch 6/10
23/23 [==============================] - 1s 31ms/step - loss: 0.8606 - categorical_accuracy: 0.6747 - val_loss: 0.9434 - val_categorical_accuracy: 0.6458
Epoch 7/10

In [ ]:
# How big is this model?
basic_conv_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_98 (Conv2D)          (None, 80, 80, 16)        448       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 40, 40, 16)       0         
 2D)                                                             
                                                                 
 conv2d_99 (Conv2D)          (None, 40, 40, 24)        3480      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 20, 20, 24)       0         
 2D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 9600)              0         
                                                                 
 dense_12 (Dense)            (None, 192)              

### Questions
1. How did the network do? What were the final training and validation accuracies?
**The network did work very well as the training accuracy and validation accuracies were comparatively higher than the previous network. The final training accuracy was 77% whereas the validation accuracy was 67%.**

2. Is there evidence of overfitting?
**Yes, we can see a clear evidence of overfitting, my training accuracy was higher than the validation accuracy from Epoch 4 through Epoch10. The training accuracies from Epoch 4 through Epoch 10 were (60%, 63%,67% 70%, 73%,76%,77%)respectively compared to their validation accuracy of(55%,50%,64% 57%,64%,59%,67%)**
3. What do convolutional networks do to improve learning?
**The convolutional networks use small filter than the input, allowing it to be multipled various times by the input array. The small filter is designed to detect a particular feature anywhere in the image that ultimately creates a feature map and helps to improve learning. Other than having small filter, convolutional networks also have multiple layers.**

## Augmented Convolutional Neural Network

Build the network and the train the model using the cells below.

In [ ]:
# A convolutional neural network with some features to combat overfitting
# Data augmentation spices up the input by applying random transformations
# Dropout nulls out neurons randomly forcing the network to develop
# redundancy and thus fit less to noise
aug_conv_model = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(24, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(192, activation='relu'),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(num_classes, activation='softmax'),
])
aug_conv_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
# Train the model
# Watch the training and validation accuracy over time
# A notably higher training accuracy means overfitting
epochs=10
history = aug_conv_model.fit(
  train,
  validation_data=validation,
  epochs=epochs
)

Epoch 1/10
23/23 [==============================] - 16s 86ms/step - loss: 1.6336 - categorical_accuracy: 0.3481 - val_loss: 1.6544 - val_categorical_accuracy: 0.2343
Epoch 2/10
23/23 [==============================] - 1s 36ms/step - loss: 1.2957 - categorical_accuracy: 0.4584 - val_loss: 1.3815 - val_categorical_accuracy: 0.3597
Epoch 3/10
23/23 [==============================] - 1s 36ms/step - loss: 1.1863 - categorical_accuracy: 0.5194 - val_loss: 1.1712 - val_categorical_accuracy: 0.5204
Epoch 4/10
23/23 [==============================] - 1s 35ms/step - loss: 1.1367 - categorical_accuracy: 0.5416 - val_loss: 1.1514 - val_categorical_accuracy: 0.5395
Epoch 5/10
23/23 [==============================] - 1s 38ms/step - loss: 1.1039 - categorical_accuracy: 0.5654 - val_loss: 1.1051 - val_categorical_accuracy: 0.5668
Epoch 6/10
23/23 [==============================] - 1s 37ms/step - loss: 1.0720 - categorical_accuracy: 0.5691 - val_loss: 1.0355 - val_categorical_accuracy: 0.5995
Epoch 7/1

In [ ]:
# Evaluate its performance on the test set
print("Testing Results:")
aug_conv_model.evaluate(test)

Testing Results:
3/3 [==============================] - 1s 189ms/step - loss: 0.8726 - categorical_accuracy: 0.6403


[0.8726206421852112, 0.640326976776123]

### Questions
1. How did the network do? What were the final training and validation accuracies?
**The network woked fairly good but the accuracy was not that high. The final training accuracy was about 62% whereas the final validation accuracy was about 66%.**
2. Is there evidence of overfitting?
**Yes, there is a clear evidence of overfitting as my training accuracy were way higher than the validation accuracy from Epoch 1 through Epoch4 except Epoch 3. The training accuracy from Epoch 1 through Epoch 4 except Epoch 3 were (35%,46%,,54.16%) respectively compared to the validation accuracy of (23%, 36%,53.9%).**
3. Is the overfitting reduced compared to the previous network?
**Yes. the overfitting reduced compared to the Basic Convolutional Network as, in the basic convolutional network we saw a lot of overfittings compared to this data. The previous network had 7 overfittings whereas the new one has only 3 overfittings.**
4. How did the network do on the test data that it hadn't seen before? Was the accuracy comparable?
**On the test data, the network gave us a loss of about 87.26%, and a accuracy of 64% however it did not gave us the validation accuracy. The loss that we got from the test data was low about 87.26% compared to what we got previously of 96.6%. The accuracy however, were only slightly different, we got a slightly higher accuracy of 64% compared to that of 62.6% causing a difference of about 1.4%.**



## Network with Pre-trained Feature Extractors

Build the network and the train the model using the cells below.

In [ ]:
# Load in the pre-trained model
# Note that include_top=False means we are including just
# the feature extractors and not the classification layers
pre_trained_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(img_size, img_size, 3), include_top=False)

# Make sure we don't try to train these massive layers
pre_trained_model.trainable = False

# Build the model as before
pre_trained_model = tf.keras.Sequential([
  pre_trained_model,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(192, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax'),
])
pre_trained_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])



87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
# Train the model
# Watch the training and validation accuracy over time
# A notably higher training accuracy means overfitting
epochs=5
history = pre_trained_model.fit(
  train,
  validation_data=validation,
  epochs=epochs
)

Epoch 1/5
23/23 [==============================] - 13s 283ms/step - loss: 1.5481 - categorical_accuracy: 0.4864 - val_loss: 0.9606 - val_categorical_accuracy: 0.6294
Epoch 2/5
23/23 [==============================] - 2s 93ms/step - loss: 0.9107 - categorical_accuracy: 0.6553 - val_loss: 0.9560 - val_categorical_accuracy: 0.6458
Epoch 3/5
23/23 [==============================] - 2s 93ms/step - loss: 0.7159 - categorical_accuracy: 0.7384 - val_loss: 0.8930 - val_categorical_accuracy: 0.6621
Epoch 4/5
23/23 [==============================] - 2s 94ms/step - loss: 0.5871 - categorical_accuracy: 0.7830 - val_loss: 0.9716 - val_categorical_accuracy: 0.6294
Epoch 5/5
23/23 [==============================] - 2s 93ms/step - loss: 0.4839 - categorical_accuracy: 0.8338 - val_loss: 0.8398 - val_categorical_accuracy: 0.6703


In [ ]:
pre_trained_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 1, 1, 2048)        21802784  
                                                                 
 flatten_7 (Flatten)         (None, 2048)              0         
                                                                 
 dense_16 (Dense)            (None, 192)               393408    
                                                                 
 dense_17 (Dense)            (None, 5)                 965       
                                                                 
Total params: 22,197,157
Trainable params: 394,373
Non-trainable params: 21,802,784
_________________________________________________________________


### Questions
1. How did the network do? What were the final training and validation accuracies?
**The network worked pretty. The final training accuracy was 83% whereas the validation accuracy was 67%.**
2. Is there evidence of overfitting?
**Yes, there is a clear evidence of overfitting. As my training accuracy was much higher compared to my validation accuracy from Epoch 2 through Epoch 5. My training accuracy from Epoch 2 through Epoch 5 were (65%, 73.8%, 78.3%, 83.3%) respectively compared to validation accuracy of (64.5%,66%,62.9%, 67%)respectively.**
3. Why might you include pre-trained layers? Are they improving performance?
**We include pre-trained layers because it converges fastly thus saves time, and there is no need to create neural networks everytime and hence is more efficient. Yes, using the pre-trained layers did improve my performance. I got my highest training accuracy of 83.38% compared to other models.**
4. How does the number of parameters (weights) compare to the previous networks (Hint: look at the summaries).
**The number of parameters are way higher than the previous networks. The number of parameters that we have in this network is 22,197,157 compared to the previous ones of 3,699,269, and 1,848,285. We can also notice that in the previous network all of the parameters were trainable leaving the non trainable to be 0. However, in this network only 394,373 parameters are trainable and 21,802,784 parameters were not trainable.**
